# Augmenting the Linear Model With a Low-Rate Camera

## Imports

In [ ]:
import sys
sys.path.insert(0, "..")  # Add parent directory for module imports

import numpy as np
import matplotlib.pyplot as plt
import control as ctl
from kalman import KalmanFilter
from lqr_controller import SwitchingLQRController
from simulation import TankDriveSimulation, discretize_system

## Parameters

In [ ]:
# Physical parameters
L = 0.5       # wheelbase [m]
tau = 0.2     # wheel speed time constant [s]
v0 = 0.5      # linearization forward speed [m/s]

# State: [x, y, theta, v_l, v_r]
n_states = 5
n_inputs = 2
n_outputs = 4   # IMU/encoders: [theta, theta_dot, v_l, v_r]

# Simulation parameters
T_final = 15.0
dt = 0.02   # 50 Hz control rate
N = int(T_final / dt)
t = np.arange(N) * dt
random_seed = 42
random_seed_camera = 123

# Noise covariances
Q = np.diag([1e-5, 1e-5, 1e-6, 1e-4, 1e-4])  # process noise
R = np.diag([1e-3, 1e-2, 1e-3, 1e-3])  # IMU/encoder measurement noise

# Initial state covariance
P0 = 0.1

# LQR weight matrices
Q_lqr = np.diag([10.0, 10.0, 5.0, 0.1, 0.1])
R_lqr = np.diag([1.0, 1.0])

# Control parameters
u_max = 2.0

# Camera/vision parameters
R_camera = 0.05  # camera position measurement noise [m] (5 cm standard deviation)
camera_rate = 5.0  # Hz (much lower than 50 Hz control rate)
camera_dt = 1.0 / camera_rate
camera_update_interval = int(camera_dt / dt)  # update every N timesteps

# Target parameters
x_target = 5.0
y_target = 3.0
target_threshold = 0.5

# Reference states
x_ref_pos = np.array([x_target, y_target, 0.0, v0, v0])
x_ref_stop = np.array([x_target, y_target, 0.0, 0.0, 0.0])

## System Setup

In [ ]:
# Create linearized system
A = np.array([
    [0, 0,   0,      0.5,   0.5],
    [0, 0,   0.5*v0,  0.0,   0.0],
    [0, 0,   0,     -1/L,   1/L],
    [0, 0,   0,  -1.0/tau,  0.0],
    [0, 0,   0,      0.0, -1.0/tau],
])

B = np.array([
    [0.0,      0.0],
    [0.0,      0.0],
    [0.0,      0.0],
    [1.0/tau,  0.0],
    [0.0,  1.0/tau],
])

# Measurement matrix: [theta, theta_dot, v_l, v_r]
C = np.array([
    [0, 0, 1,     0,    0],      # magnetometer: theta
    [0, 0, 0, -1/L, 1/L],        # gyro: theta_dot
    [0, 0, 0,     1,    0],      # left encoder: v_l
    [0, 0, 0,     0,    1],      # right encoder: v_r
])
D = np.zeros((n_outputs, n_inputs))

# Discretize
Ad, Bd, Cd, Dd = discretize_system(A, B, C, D, dt)

# Initialize LQR controller
lqr = SwitchingLQRController(
    Ad, Bd, Q_lqr, R_lqr,
    x_ref_pos=x_ref_pos,
    x_ref_stop=x_ref_stop,
    target_pos=[x_target, y_target],
    target_threshold=target_threshold,
    u_max=u_max
)

print("System initialized")
print(f"Control rate: {1/dt:.0f} Hz")
print(f"Camera rate: {camera_rate:.0f} Hz")
print(f"Camera updates every {camera_update_interval} control steps ({camera_dt:.3f} s)")

## Baseline: Dead Reckoning Only

First, run a simulation with only IMU and wheel encoders (no position measurements).

In [ ]:
# Initialize Kalman filter (no camera)
x0 = np.array([0.0, 0.0, 0.0, 0.0, 0.0])
P_init = np.eye(n_states) * P0
kf_baseline = KalmanFilter(Ad, Bd, Cd, Q, R, x0=x0, P0=P_init)

# Initialize baseline simulation
system_matrices = {'Ad': Ad, 'Bd': Bd, 'Cd': Cd}
sim_baseline = TankDriveSimulation(
    system_matrices=system_matrices,
    filter_obj=kf_baseline,
    controller_obj=lqr,
    Q=Q,
    R=R,
    dt=dt,
    N=N,
    random_seed=random_seed
)

# Set initial conditions
sim_baseline.set_initial_conditions(x0_true=x0, x0_hat=x0)

# Run baseline simulation
results_baseline = sim_baseline.run()

print("Baseline (dead reckoning) simulation complete!")

# Extract results
x_true = results_baseline['x_true']
x_hat = results_baseline['x_hat']
P_hist = results_baseline['P_hist']

## Camera-Augmented Navigation

Now add intermittent position measurements from a camera/vision system.

### Time-Varying Measurement Model

- **High-rate (50 Hz)**: $\mathbf{y} = [\theta, \dot{\theta}, v_l, v_r]^T$ (IMU/encoders)
- **Low-rate (5 Hz)**: $\mathbf{y} = [\theta, \dot{\theta}, v_l, v_r, x, y]^T$ (IMU/encoders + camera)

The Kalman filter automatically handles the different measurement dimensions and noise levels.

In [ ]:
# Augmented measurement matrix for camera updates
n_outputs_cam = 6  # [theta, theta_dot, v_l, v_r, x, y]
C_with_camera = np.array([
    [0, 0, 1,     0,    0],      # magnetometer: theta
    [0, 0, 0, -1/L, 1/L],        # gyro: theta_dot
    [0, 0, 0,     1,    0],      # left encoder: v_l
    [0, 0, 0,     0,    1],      # right encoder: v_r
    [1, 0, 0,     0,    0],      # camera: x
    [0, 1, 0,     0,    0],      # camera: y
])
Cd_cam = C_with_camera

# Measurement noise with camera
R_with_cam = np.diag([1e-3, 1e-2, 1e-3, 1e-3, R_camera**2, R_camera**2])

print(f"Camera measurement noise: {R_camera*1000:.1f} mm (1σ)")
print(f"\nAugmented measurement matrix C_cam:")
print(Cd_cam)

In [ ]:
# Initialize Kalman filter with camera
x0_cam = np.array([0.0, 0.0, 0.0, 0.0, 0.0])
P_init_cam = np.eye(n_states) * P0
kf_cam = KalmanFilter(Ad, Bd, Cd, Q, R, x0=x0_cam, P0=P_init_cam)

# Storage arrays (manual since we need custom measurement logic)
x_true_cam = np.zeros((n_states, N))
y_meas_cam = np.zeros((n_outputs_cam, N))
u_control_cam = np.zeros((N, n_inputs))
x_hat_cam = np.zeros((n_states, N))
P_hist_cam = np.zeros((n_states, N))

# Initial conditions
x_true_cam[:, 0] = x0_cam
x_hat_cam[:, 0] = kf_cam.get_state()
P_hist_cam[:, 0] = kf_cam.get_uncertainty()**2

# Noise generators
Q_chol = np.linalg.cholesky(Q)
R_chol_cam = np.linalg.cholesky(R_with_cam)
rng_cam = np.random.default_rng(random_seed_camera)

# Simulation loop with camera updates (custom time-varying measurement model)
for k in range(N - 1):
    # Generate full measurement (always includes position from true state)
    v_k_cam = R_chol_cam @ rng_cam.standard_normal(n_outputs_cam)
    y_meas_cam[:, k] = Cd_cam @ x_true_cam[:, k] + v_k_cam
    
    # Kalman filter predict
    kf_cam.predict(u_control_cam[k-1, :] if k > 0 else np.zeros(n_inputs))
    
    # Kalman filter update (time-varying measurement model)
    if k % camera_update_interval == 0:
        # Camera update available - use full measurement
        kf_cam.C = Cd_cam
        kf_cam.R = R_with_cam
        kf_cam.update(y_meas_cam[:, k])
        # Restore original matrices
        kf_cam.C = Cd
        kf_cam.R = R
    else:
        # Only IMU/encoder measurements
        kf_cam.update(y_meas_cam[:4, k])
    
    x_hat_cam[:, k] = kf_cam.get_state()
    P_hist_cam[:, k] = kf_cam.get_uncertainty()**2

    # Control (same as baseline)
    u_control_cam[k, :] = lqr.compute_control(x_hat_cam[:, k])

    # True system
    w_k = Q_chol @ rng_cam.standard_normal(n_states)
    x_true_cam[:, k+1] = Ad @ x_true_cam[:, k] + Bd @ u_control_cam[k, :] + w_k

# Final step
y_meas_cam[:, -1] = Cd_cam @ x_true_cam[:, -1] + R_chol_cam @ rng_cam.standard_normal(n_outputs_cam)
kf_cam.predict(u_control_cam[-2, :])

if (N-1) % camera_update_interval == 0:
    kf_cam.C = Cd_cam
    kf_cam.R = R_with_cam
    kf_cam.update(y_meas_cam[:, -1])
    kf_cam.C = Cd
    kf_cam.R = R
else:
    kf_cam.update(y_meas_cam[:4, -1])

x_hat_cam[:, -1] = kf_cam.get_state()
P_hist_cam[:, -1] = kf_cam.get_uncertainty()**2

print("Camera-augmented simulation complete!")

## Results: Comparison

Compare dead reckoning vs camera-augmented navigation.

In [ ]:
# Compute position errors
pos_error_baseline = np.sqrt((x_true[0, :] - x_hat[0, :])**2 + (x_true[1, :] - x_hat[1, :])**2)
pos_error_camera = np.sqrt((x_true_cam[0, :] - x_hat_cam[0, :])**2 + (x_true_cam[1, :] - x_hat_cam[1, :])**2)

# Camera update times for plotting
camera_times = t[::camera_update_interval]

fig, axes = plt.subplots(3, 1, figsize=(12, 10))

# Position uncertainty comparison
axes[0].plot(t, np.sqrt(P_hist[0, :]), 'b-', linewidth=2, label='σ_x (dead reckoning)')
axes[0].plot(t, np.sqrt(P_hist[1, :]), 'r-', linewidth=2, label='σ_y (dead reckoning)')
axes[0].plot(t, np.sqrt(P_hist_cam[0, :]), 'b--', linewidth=2, alpha=0.7, label='σ_x (with camera)')
axes[0].plot(t, np.sqrt(P_hist_cam[1, :]), 'r--', linewidth=2, alpha=0.7, label='σ_y (with camera)')

# Mark camera update times
for ct in camera_times:
    axes[0].axvline(ct, color='gray', linestyle=':', alpha=0.3, linewidth=1)

axes[0].set_ylabel('Position uncertainty [m]')
axes[0].set_xlabel('time [s]')
axes[0].grid(True, alpha=0.3)
axes[0].legend(loc='upper left')
axes[0].set_title('Position Uncertainty: Dead Reckoning vs Camera-Aided')

# Trajectory comparison
axes[1].plot(x_true[0, :], x_true[1, :], 'b-', linewidth=2, label='True (dead reckoning)')
axes[1].plot(x_hat[0, :], x_hat[1, :], 'g--', linewidth=2, label='Estimate (dead reckoning)')
axes[1].plot(x_true_cam[0, :], x_true_cam[1, :], 'b-', linewidth=2, alpha=0.5, label='True (with camera)')
axes[1].plot(x_hat_cam[0, :], x_hat_cam[1, :], 'm--', linewidth=2, label='Estimate (with camera)')
axes[1].plot(x_target, y_target, 'rx', markersize=15, markeredgewidth=3, label='Target')
axes[1].set_xlabel('x [m]')
axes[1].set_ylabel('y [m]')
axes[1].axis('equal')
axes[1].grid(True, alpha=0.3)
axes[1].legend()
axes[1].set_title('Trajectory Comparison')

# Position error comparison
axes[2].plot(t, pos_error_baseline, 'b-', linewidth=2, label='Position error (dead reckoning)')
axes[2].plot(t, pos_error_camera, 'r-', linewidth=2, label='Position error (with camera)')

# Mark camera update times
for ct in camera_times:
    axes[2].axvline(ct, color='gray', linestyle=':', alpha=0.3, linewidth=1)

axes[2].set_ylabel('Position error [m]')
axes[2].set_xlabel('time [s]')
axes[2].grid(True, alpha=0.3)
axes[2].legend()
axes[2].set_title('Actual Position Estimation Error')

plt.tight_layout()
plt.show()

# Print statistics
print("\n" + "="*60)
print("COMPARISON: Dead Reckoning vs Camera-Aided Navigation")
print("="*60)
print(f"\nFinal position uncertainty (1σ):")
print(f"  Dead reckoning: x={np.sqrt(P_hist[0, -1]):.4f} m, y={np.sqrt(P_hist[1, -1]):.4f} m")
print(f"  With camera:    x={np.sqrt(P_hist_cam[0, -1]):.4f} m, y={np.sqrt(P_hist_cam[1, -1]):.4f} m")
print(f"\nMean position error:")
print(f"  Dead reckoning: {np.mean(pos_error_baseline):.4f} m")
print(f"  With camera:    {np.mean(pos_error_camera):.4f} m")
print(f"\nMax position error:")
print(f"  Dead reckoning: {np.max(pos_error_baseline):.4f} m")
print(f"  With camera:    {np.max(pos_error_camera):.4f} m")
print(f"\nUncertainty reduction: {np.sqrt(P_hist[0, -1])/np.sqrt(P_hist_cam[0, -1]):.1f}x")